In [0]:
dados = [
  ("Antonio", "Joao", "Analista", "Brasileira", "M", 10000),
  ("Andre", "Luiz", "Operario", "Brasileira", "M", 2000),
  ("Maria", "Jose", "Atriz", "Portuguesa", "F", 15000),
  ("Juliana", "Rosa", "Medica", "Italiana", "F", 20000),
  ("Roberto", "Carlos", "Dentista", "Americana", "M", 20000)
]

colunas = ["Primeiro_Nome", "Ultimo_Nome","Trabalho", "Nacionalidade", "Genero", "Salario"]

datafparquet=spark.createDataFrame(dados, colunas)

datafparquet.show()

+-------------+-----------+--------+-------------+------+-------+
|Primeiro_Nome|Ultimo_Nome|Trabalho|Nacionalidade|Genero|Salario|
+-------------+-----------+--------+-------------+------+-------+
|      Antonio|       Joao|Analista|   Brasileira|     M|  10000|
|        Andre|       Luiz|Operario|   Brasileira|     M|   2000|
|        Maria|       Jose|   Atriz|   Portuguesa|     F|  15000|
|      Juliana|       Rosa|  Medica|     Italiana|     F|  20000|
|      Roberto|     Carlos|Dentista|    Americana|     M|  20000|
+-------------+-----------+--------+-------------+------+-------+



In [0]:
#criando o arquivo parquet
datafparquet.write.parquet("/FileStore/tables/parquet/pessoal_new.parquet")

In [0]:
#Permite uma atualização do arquivo parquet
datafparquet.write.mode('overwrite').parquet('/FileStore/tables/parquet/pessoal_new.parquet')

In [0]:
#Realizando uma atualização do arquivo parquet
datafleitura=spark.read.parquet("/FileStore/tables/parquet/pessoal_new.parquet")
datafleitura.show()

+-------------+-----------+--------+-------------+------+-------+
|Primeiro_Nome|Ultimo_Nome|Trabalho|Nacionalidade|Genero|Salario|
+-------------+-----------+--------+-------------+------+-------+
|      Roberto|     Carlos|Dentista|    Americana|     M|  20000|
|      Antonio|       Joao|Analista|   Brasileira|     M|  10000|
|        Andre|       Luiz|Operario|   Brasileira|     M|   2000|
|      Juliana|       Rosa|  Medica|     Italiana|     F|  20000|
|        Maria|       Jose|   Atriz|   Portuguesa|     F|  15000|
+-------------+-----------+--------+-------------+------+-------+



In [0]:
#Criando uma consulta em SQL
datafleitura.createOrReplaceTempView("Tabela_Parquet")
ResultSQL = spark.sql("select * from Tabela_Parquet where salario >= 10000 ")
ResultSQL.show()

+-------------+-----------+--------+-------------+------+-------+
|Primeiro_Nome|Ultimo_Nome|Trabalho|Nacionalidade|Genero|Salario|
+-------------+-----------+--------+-------------+------+-------+
|      Roberto|     Carlos|Dentista|    Americana|     M|  20000|
|      Antonio|       Joao|Analista|   Brasileira|     M|  10000|
|      Juliana|       Rosa|  Medica|     Italiana|     F|  20000|
|        Maria|       Jose|   Atriz|   Portuguesa|     F|  15000|
+-------------+-----------+--------+-------------+------+-------+



In [0]:
#Particionando os dados em um arquivo parquet
datafparquet.write.partitionBy("Nacionalidade","salario").mode("overwrite").parquet("/FileStore/tables/parquet/pessoal_new.parquet")

In [0]:
#Lendo o aquivo participonado do parquet
datafnacional=spark.read.parquet("/FileStore/tables/parquet/pessoal_new.parquet/Nacionalidade=Brasileira")
datafnacional.show(truncate=False)

+-------------+-----------+--------+------+-------+
|Primeiro_Nome|Ultimo_Nome|Trabalho|Genero|salario|
+-------------+-----------+--------+------+-------+
|Antonio      |Joao       |Analista|M     |10000  |
|Andre        |Luiz       |Operario|M     |2000   |
+-------------+-----------+--------+------+-------+



In [0]:
#Consultando diretamente o arquivo parquet particionado via SQL
spark.sql("CREATE OR REPLACE TEMPORARY VIEW Cidadao USING parquet OPTIONS (path \"/FileStore/tables/parquet/pessoal_new.parquet/Nacionalidade=Brasileira\")")
spark.sql("SELECT * FROM Cidadao where Ultimo_nome='Joao'").show()

+-------------+-----------+--------+------+-------+
|Primeiro_Nome|Ultimo_Nome|Trabalho|Genero|salario|
+-------------+-----------+--------+------+-------+
|      Antonio|       Joao|Analista|     M|  10000|
+-------------+-----------+--------+------+-------+

